In [26]:
from langchain.document_loaders import JSONLoader

jq_schema = '.insights[]'

loader = JSONLoader(file_path='./DB.json', jq_schema=jq_schema, text_content=False)
docs = loader.load()

In [27]:
type(docs)

list

In [28]:
docs

[Document(page_content='{"source": {"type": "person", "name": "Albert Einstein", "profession": "Physicist"}, "insight": "Life is like riding a bicycle. To keep your balance, you must keep moving.", "category": "Motivation", "utilization": "Use this insight to remind yourself that continuous effort and progress are essential in overcoming challenges and maintaining stability in life."}', metadata={'source': '/Users/gangjimin/Documents/GitHub/Trivialog/Compass_demo/DB.json', 'seq_num': 1}),
 Document(page_content='{"source": {"type": "person", "name": "Maya Angelou", "profession": "Poet"}, "insight": "You will face many defeats in life, but never let yourself be defeated.", "category": "Resilience", "utilization": "Apply this insight by viewing setbacks as opportunities to learn and grow, maintaining a positive mindset, and persevering through difficult times."}', metadata={'source': '/Users/gangjimin/Documents/GitHub/Trivialog/Compass_demo/DB.json', 'seq_num': 2}),
 Document(page_conten

In [37]:
import json

def format_document_content(document_dict):
    if document_dict['source']['type'] == 'person':
        source = f"{document_dict['source']['name']}, {document_dict['source']['profession']}"
    else:
        source = f"{document_dict['source']['title']} by {document_dict['source']['author']}"
    
    document_content = f"""
Source of Insight: {source}.
Content of insight: {document_dict['insight']}.
Useful insight in: {document_dict['category']}.
That insight could be helpful in the following way:
{document_dict['utilization']}
"""
    return document_content

with open("DB.txt", "a") as file:
    for i in range(len(docs)):
        document = docs[i].page_content
        document_dict = json.loads(document)
        data = format_document_content(document_dict)
        file.write(data + '\n')

In [29]:
docs[0]

Document(page_content='{"source": {"type": "person", "name": "Albert Einstein", "profession": "Physicist"}, "insight": "Life is like riding a bicycle. To keep your balance, you must keep moving.", "category": "Motivation", "utilization": "Use this insight to remind yourself that continuous effort and progress are essential in overcoming challenges and maintaining stability in life."}', metadata={'source': '/Users/gangjimin/Documents/GitHub/Trivialog/Compass_demo/DB.json', 'seq_num': 1})

In [30]:
docs_0 = docs[0].page_content

In [31]:
docs_0

'{"source": {"type": "person", "name": "Albert Einstein", "profession": "Physicist"}, "insight": "Life is like riding a bicycle. To keep your balance, you must keep moving.", "category": "Motivation", "utilization": "Use this insight to remind yourself that continuous effort and progress are essential in overcoming challenges and maintaining stability in life."}'

In [32]:
import json 

docs_0_dict = json.loads(docs_0)

In [33]:
docs_0_dict

{'source': {'type': 'person',
  'name': 'Albert Einstein',
  'profession': 'Physicist'},
 'insight': 'Life is like riding a bicycle. To keep your balance, you must keep moving.',
 'category': 'Motivation',
 'utilization': 'Use this insight to remind yourself that continuous effort and progress are essential in overcoming challenges and maintaining stability in life.'}

In [34]:
docs_0_dict['insight']

'Life is like riding a bicycle. To keep your balance, you must keep moving.'

In [35]:
docs_0_dict['utilization']

'Use this insight to remind yourself that continuous effort and progress are essential in overcoming challenges and maintaining stability in life.'

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

class CustomTextSplitter(CharacterTextSplitter):
    def __init__(self, delimiter='/', **kwargs):
        super().__init__(**kwargs)
        self.delimiter = delimiter

    def split_text(self, text: str):
        return text.split(self.delimiter)

In [2]:

file_path = './DB_Book_and_Famous.txt'
loader = TextLoader(file_path)

splitter = CustomTextSplitter(delimiter='\n\n')

documents = loader.load()
print(documents)
print(len(documents))

# for document in documents:
#   print(document.page_content)

document = splitter.split_documents(documents)
print(len(document))
for chunk in document:
  print(chunk)

# for document in documents:
#     chunks = splitter.split_text(document.page_content)

# print(chunks)
# print(type(chunks))
# print(len(chunks))

[Document(page_content="Source of Insight: Albert Einstein, Physicist.\nContent of insight: Life is like riding a bicycle. To keep your balance, you must keep moving..\nUseful insight in: Motivation.\nThat insight could be helpful in the following way:\nUse this insight to remind yourself that continuous effort and progress are essential in overcoming challenges and maintaining stability in life.  \n\n\nSource of Insight: Maya Angelou, Poet.\nContent of insight: You will face many defeats in life, but never let yourself be defeated..\nUseful insight in: Resilience.\nThat insight could be helpful in the following way:\nApply this insight by viewing setbacks as opportunities to learn and grow, maintaining a positive mindset, and persevering through difficult times. \n\n\nSource of Insight: The Power of Now by Eckhart Tolle.\nContent of insight: Realize deeply that the present moment is all you ever have..\nUseful insight in: Mindfulness.\nThat insight could be helpful in the following wa

In [3]:
from langchain.embeddings import OpenAIEmbeddings

In [4]:
embed_model = OpenAIEmbeddings(model='text-embedding-3-large')

/Users/gangjimin/Documents/GitHub/Trivialog/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(document, embed_model)

In [6]:
print(type(vector_index))

<class 'langchain_community.vectorstores.faiss.FAISS'>


In [7]:
print(vector_index)

In [8]:
print(len(vector_index.index_to_docstore_id))

38


In [9]:
print(type(vector_index.index))

<class 'faiss.swigfaiss.IndexFlatL2'>


In [10]:
retriever = vector_index.as_retriever(
  search_kwargs={'k': 3}
)

In [11]:
query = 'I am tired.'
retrieved_docs = retriever.invoke(query)

In [12]:
for doc in retrieved_docs:
  print(doc)

page_content='\nSource of Insight: Pride and Prejudice by Jane Austen.\nContent of insight: I declare after all there is no enjoyment like reading! How much sooner one tires of anything than of a book!.\nUseful insight in: Reading.\nThat insight could be helpful in the following way:\nEmbrace reading as a source of joy and enrichment, regularly engaging with books to broaden your perspective and knowledge.  ' metadata={'source': './DB_Book_and_Famous.txt'}
page_content='\nSource of Insight: Maya Angelou, Poet.\nContent of insight: You will face many defeats in life, but never let yourself be defeated..\nUseful insight in: Resilience.\nThat insight could be helpful in the following way:\nApply this insight by viewing setbacks as opportunities to learn and grow, maintaining a positive mindset, and persevering through difficult times. ' metadata={'source': './DB_Book_and_Famous.txt'}
page_content='\nSource of Insight: The Great Gatsby by F. Scott Fitzgerald.\nContent of insight: So we bea

In [13]:
print(vector_index.index.ntotal)

38


In [14]:
store_path = './db/faiss_index'
vector_index.save_local(store_path)

In [43]:
vector_index = FAISS.load_local(store_path, embed_model, allow_dangerous_deserialization=True)

In [44]:
vector_index.index.ntotal

38

In [58]:
retriever = vector_index.as_retriever(
  search_kwargs={'k': 3}
)

retriever_threshold = vector_index.as_retriever(
  search_type='similarity_score_threshold',
  search_kwargs={'score_threshold': 0.4}
)

In [62]:
retrieved_docs = retriever.get_relevant_documents(query)
retrieved_docs_threshold = retriever_threshold.get_relevant_documents(query)

/Users/gangjimin/Documents/GitHub/Trivialog/venv/lib/python3.11/site-packages/langchain_core/vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="\nSource of Insight: Nelson Mandela, Political Leader.\nContent of insight: It always seems impossible until it's done..\nUseful insight in: Perseverance.\nThat insight could be helpful in the following way:\nUse this insight to push through challenging tasks by remembering that many great achievements once seemed impossible. ", metadata={'source': './DB_Book_and_Famous.txt'}), -0.12184938791642219), (Document(page_content='\nSource of Insight: Thinking, Fast and Slow by Daniel Kahneman.\nContent of insight: Nothing in life is as important as you think it is, while you are thinking about it..\nUseful insight in: Perspective.\nThat insight could be helpful in the following way:\nMaintain perspective by recognizing that immediate concerns often seem more significant than they truly are, helpin

In [61]:
print(retrieved_docs)
print(type(retrieved_docs))

[Document(page_content="\nSource of Insight: Nelson Mandela, Political Leader.\nContent of insight: It always seems impossible until it's done..\nUseful insight in: Perseverance.\nThat insight could be helpful in the following way:\nUse this insight to push through challenging tasks by remembering that many great achievements once seemed impossible. ", metadata={'source': './DB_Book_and_Famous.txt'}), Document(page_content='\nSource of Insight: Thinking, Fast and Slow by Daniel Kahneman.\nContent of insight: Nothing in life is as important as you think it is, while you are thinking about it..\nUseful insight in: Perspective.\nThat insight could be helpful in the following way:\nMaintain perspective by recognizing that immediate concerns often seem more significant than they truly are, helping you to manage stress and make better decisions.  ', metadata={'source': './DB_Book_and_Famous.txt'}), Document(page_content='Source of Insight: Albert Einstein, Physicist.\nContent of insight: Lif

In [50]:
for doc in retrieved_docs:
  print(doc.page_content)
  print(type(doc))


Source of Insight: Nelson Mandela, Political Leader.
Content of insight: It always seems impossible until it's done..
Useful insight in: Perseverance.
That insight could be helpful in the following way:
Use this insight to push through challenging tasks by remembering that many great achievements once seemed impossible. 
<class 'langchain_core.documents.base.Document'>

Source of Insight: Thinking, Fast and Slow by Daniel Kahneman.
Content of insight: Nothing in life is as important as you think it is, while you are thinking about it..
Useful insight in: Perspective.
That insight could be helpful in the following way:
Maintain perspective by recognizing that immediate concerns often seem more significant than they truly are, helping you to manage stress and make better decisions.  
<class 'langchain_core.documents.base.Document'>
Source of Insight: Albert Einstein, Physicist.
Content of insight: Life is like riding a bicycle. To keep your balance, you must keep moving..
Useful insight